In [0]:
silver_cust_table = "zions_schema.silver_cust"
dim_table = "zions_schema.dim_customer_scd2" 

In [0]:
df_events = spark.table(silver_cust_table)

In [0]:
%sql
select * from zions_schema.silver_cust

In [0]:
spark.sql(f"""CREATE OR REPLACE TEMP VIEW vw_events_dedup AS
SELECT
  cust_id,
  event_type,
  event_ts_parsed,            
  first_name,
  last_name,
  email,
  phone,
  addr_city,
  addr_state,
  addr_postal,
  ingest_ts,
  source_file,
  ROW_NUMBER() OVER (
    PARTITION BY cust_id, event_ts_parsed, first_name, last_name, email, phone, addr_city, addr_state, addr_postal
    ORDER BY ingest_ts DESC NULLS LAST
  ) AS rn
FROM {silver_cust_table}
""")

In [0]:
%sql
CREATE OR REPLACE TEMP VIEW vw_events_for_scd AS
SELECT *
FROM vw_events_dedup
WHERE rn = 1;

In [0]:
%sql
select * from vw_events_dedup 
ORDER BY cust_id, event_ts_parsed, ingest_ts;

In [0]:
%sql
CREATE OR REPLACE TEMP VIEW vw_dim_reconstructed AS
SELECT
  cust_id,
  first_name,
  last_name,
  email,
  phone,
  addr_city,
  addr_state,
  addr_postal,
  event_ts_parsed AS eff_start,
  LEAD(event_ts_parsed) OVER (PARTITION BY cust_id ORDER BY event_ts_parsed, ingest_ts) AS eff_end,
  CASE 
    WHEN LEAD(event_ts_parsed) OVER (PARTITION BY cust_id ORDER BY event_ts_parsed, ingest_ts) IS NULL 
      THEN true 
    ELSE false 
  END AS is_current,
  event_type,
  ingest_ts,
  source_file

FROM vw_events_for_scd
ORDER BY cust_id, eff_start;

In [0]:
%sql
select * from vw_dim_reconstructed

In [0]:
%sql
CREATE OR REPLACE TEMP VIEW vw_dim_final AS
SELECT
 row_number() OVER (ORDER BY cust_id, eff_start) AS scd_row_id,
  cust_id,
  first_name,
  last_name,
  email,
  phone,
  addr_city,
  addr_state,
  addr_postal,
  eff_start,
  eff_end,
  is_current,
  event_type,
  ingest_ts,
  source_file
FROM vw_dim_reconstructed;

In [0]:
%sql
SELECT * FROM vw_dim_final

In [0]:
dim_df = spark.sql("SELECT * FROM vw_dim_final")


(dim_df.write
  .mode("overwrite")
  .option("overwriteSchema", "true")
  .saveAsTable(dim_table))

In [0]:
spark.sql(f"SELECT * FROM {dim_table} ORDER BY cust_id, eff_start").show(20, False)